In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.core.frame import DataFrame

In [2]:
#引入变数名称整理后的Excel便于对照含义
translation = pd.read_csv('translation.csv')
translation

,表頭,中文名稱,表頭.1,中文名稱.1,表頭.2,中文名稱.2,表頭.3,中文名稱.3
0,AGE,年齡,cucah,本月借現金額,inamt,本月分期消費金額,trans_no,交易編號
1,alert_key,alert主鍵,cucsm,本月消費金額,info_asset_code,資訊資產代號,tx_amt,交易金額
2,amt,交易金額-台幣,cur_type,消費地幣別,lupay,上月繳款總額,tx_date,交易日期
3,ATM,是否為實體ATM交易,cust_id,顧客編號,occupation_code,職業,tx_time,交易時間
4,byymm,帳務年月,cycam,信用額度,risk_rank,風險等級,tx_type,交易類別
5,clamt,本月分期預借現金金額,date,消費日期,sar_flag,alert主鍵報SAR與否,txbranch,分行代碼
6,country,消費地國別,debit_credit,借貸別,total_asset,行內總資產,usgam,已使用額度
7,cross_bank,是否為跨行交易,exchg_rate,匯率,trade_amount_usd,交易金額(折合美金),NaN,NaN
8,csamt,本月預借現金金額,fiscTxId,交易代碼,trans_date,外匯交易日(帳務日),NaN,NaN


In [3]:
train_x_ccba = pd.read_csv('public_train_x_ccba_full_hashed.csv')
train_x_cdtx0001 = pd.read_csv('public_train_x_cdtx0001_full_hashed.csv')
train_x_custinf = pd.read_csv('public_train_x_custinfo_full_hashed.csv')
train_x_dp = pd.read_csv('public_train_x_dp_full_hashed.csv')
train_x_remit1 = pd.read_csv('public_train_x_remit1_full_hashed.csv')
public_x_alert = pd.read_csv('public_x_alert_date.csv')
train_x_alert = pd.read_csv('train_x_alert_date.csv')
train_y_answer = pd.read_csv('train_y_answer.csv')

In [4]:
print("train_x_ccba",train_x_ccba.shape) 
print("train_x_cdtx0001",train_x_cdtx0001.shape) 
print("train_x_custinf",train_x_custinf.shape) 
print("train_x_dp",train_x_dp.shape) 
print("train_x_remit1",train_x_remit1.shape) 
print("public_x_alert",public_x_alert.shape) 
print("train_x_alert",train_x_alert.shape) 
print("train_y_answer",train_y_answer.shape) 

train_x_ccba (59075, 10)
train_x_cdtx0001 (1043014, 5)
train_x_custinf (25751, 6)
train_x_dp (1969818, 12)
train_x_remit1 (17167, 4)
public_x_alert (1845, 2)
train_x_alert (23906, 2)
train_y_answer (23906, 2)


**1.对每个子表的cust_id进行计数，用于后续匹配Y-answer的值  
可以看出重复值最高的次数为cdtx0001 countmax = 118189；dp countmax = 135100  
因此后面的重复次数设为10000 和 150000-15万次**

In [5]:
train_x_ccba_countid=pd.pivot_table(train_x_ccba,index=[u'cust_id'],aggfunc='count')
train_x_ccba_countid = train_x_ccba_countid.reset_index()#透视后变成一个column，重设为2个column
print(train_x_ccba_countid)

train_x_cdtx0001_countid=pd.pivot_table(train_x_cdtx0001,index=[u'cust_id'],aggfunc='count')
train_x_cdtx0001_countid = train_x_cdtx0001_countid.reset_index()#透视后变成一个column，重设为2个column
print(train_x_cdtx0001_countid)

train_x_custinf_countid=pd.pivot_table(train_x_custinf,index=[u'cust_id'],aggfunc='count')
train_x_custinf_countid = train_x_custinf_countid.reset_index()#透视后变成一个column，重设为2个column
print(train_x_custinf_countid)

train_x_dp_countid=pd.pivot_table(train_x_dp,index=[u'cust_id'],aggfunc='count')
train_x_dp_countid = train_x_dp_countid.reset_index()#透视后变成一个column，重设为2个column
print(train_x_dp_countid)

train_x_remit1_countid=pd.pivot_table(train_x_remit1,index=[u'cust_id'],aggfunc='count')
train_x_remit1_countid = train_x_remit1_countid.reset_index()#透视后变成一个column，重设为2个column
print(train_x_remit1_countid)

public_x_alert_countid=pd.pivot_table(public_x_alert,index=[u'alert_key'],aggfunc='count')
public_x_alert_countid = public_x_alert_countid.reset_index()#透视后变成一个column，重设为2个column
print(public_x_alert_countid)

                                                cust_id   byymm    clamt   \
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...       13       13   
1     002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...       13       13   
2     002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...       13       13   
3     0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...       13       13   
4     0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...        3        3   
...                                                 ...      ...      ...   
4740  ffa5bcc6f88df5c33e809da4daa54de0031803f37d8f11...       13       13   
4741  ffae7fef923c78298cd647a3d573e48cc1404b07ac794a...       13       13   
4742  ffdbc1a5135cb6feddf7355e5656abf6973f21639c7b9b...       13       13   
4743  ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...       13       13   
4744  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...       13       13   

       csamt    cucah    cucsm    cycam    inamt    lupay    usgam   
0    

In [6]:
print(train_x_cdtx0001_countid.loc[:,"amt"].max())
print(train_x_custinf_countid.loc[:,"AGE"].max())
print(train_x_dp_countid.loc[:,"ATM"].max())
print(train_x_remit1_countid.loc[:,"trade_amount_usd"].max())
#print(train_x_ccba_countid.loc[:,"byymm"].max())编译错误，但从Excel分析可得是13

118189
246
135100
432


**2.将train_x_alert 与 train_y_answer 按照alert key合并为一张表**

In [7]:
print(train_x_alert.head())
print(train_y_answer.head())
print(train_x_alert.shape)
print(train_y_answer.shape)

   alert_key  date
0     171189     0
1     171202     0
2     171599     0
3     171737     0
4     171142     0
   alert_key  sar_flag
0     171189         0
1     171202         0
2     171599         0
3     171737         0
4     171142         0
(23906, 2)
(23906, 2)


In [12]:
#合并train_x_alert与train_y_answer，row行数相同，how用任何形式都可以
train_x_alert_train_y_answer=pd.merge(train_x_alert,train_y_answer,how = 'outer',on = ['alert_key'],sort = False)

In [13]:
print(train_x_alert_train_y_answer)

       alert_key  date  sar_flag
0         171189     0         0
1         171202     0         0
2         171599     0         0
3         171737     0         0
4         171142     0         0
...          ...   ...       ...
23901     352132   364         0
23902     352125   364         0
23903     352080   364         0
23904     352075   364         0
23905     351927   364         0

[23906 rows x 3 columns]


**3.将合并好的alert_key和answer  同custinf 合并，便于匹配ID   
custinf里同一个ID有不同的alert_key多个  ，不同的alert_key在同一个ID下的总计数 = ID出现的次数，即一个ID一个alert_key  
在Y值上有的全部是0，有的全部是1，有点有0或1  
在answer的 23906 个alert key里，一共有不重复ID 7264个，其中sar 合计数为0的7031，为1的232，为2的1个**

In [14]:
print(train_x_custinf.head())
print(train_x_custinf.shape)

   alert_key                                            cust_id  risk_rank  \
0     352249  82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...          1   
1     352253  b212d14cb35676926682b2cf849e295d948888f556c07e...          1   
2     352254  e5b0002791c7852644a2730abeaa893cdf14a072ef7812...          1   
3     352280  74214c478dc6519fbefe4bc31693865bdcd698ab974b64...          3   
4     352282  0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...          1   

   occupation_code  total_asset  AGE  
0             19.0    1465816.0    7  
1              2.0      98177.0    2  
2             19.0    2052922.0    7  
3             15.0     201906.0    5  
4             12.0       7450.0    5  
(25751, 6)


In [15]:
#how = 'outer'（全部）left（左邊）right（右邊）inner（共有的）
#合并train_x_alert与train_y_answer与train_x_custinf==保留只有0和1的原始數據23906
train_x_alert_train_y_answer_train_x_custinf=pd.merge(train_x_alert_train_y_answer,train_x_custinf,how = 'left',on = ['alert_key'],sort = False)

In [16]:
print(train_x_alert_train_y_answer_train_x_custinf.head())
print("train_x_alert_train_y_answer_train_x_custinf",train_x_alert_train_y_answer_train_x_custinf.shape) 

   alert_key  date  sar_flag  \
0     171189     0         0   
1     171202     0         0   
2     171599     0         0   
3     171737     0         0   
4     171142     0         0   

                                             cust_id  risk_rank  \
0  972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...          1   
1  a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...          3   
2  3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...          1   
3  a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...          3   
4  a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...          3   

   occupation_code  total_asset  AGE  
0             17.0     375576.0    4  
1             12.0    2717416.0    2  
2             12.0     326517.0    4  
3             14.0    1014759.0    4  
4             12.0     241719.0    3  
train_x_alert_train_y_answer_train_x_custinf (23906, 8)


In [17]:
# 对train_x_alert_train_y_answer_train_x_custinf透视：一个ID有多个sar值
alert_answer_custinf_1=pd.pivot_table(train_x_alert_train_y_answer_train_x_custinf,index=[u'cust_id'],values=[u'sar_flag'],aggfunc = {'sum'})
alert_answer_custinf_1 = alert_answer_custinf_1.reset_index()#透视后变成一个column，重设为2个column
alert_answer_custinf_1.columns = ["cust_id","sar_flag"]#这里的sar_flag实际上是这个ID下为1的个数，值范围为0,1,2（在answer里最多出现2次）
print(alert_answer_custinf_1)

alert_answer_custinf_1['sar_flag'] == 0
fliter_1 = (alert_answer_custinf_1['sar_flag'] == 0)
alert_answer_custinf_answer0 = alert_answer_custinf_1[fliter_1]
#alert_answer_custinf_answer0 = alert_answer_custinf_answer0.reset_index()
print(alert_answer_custinf_answer0)

alert_answer_custinf_1['sar_flag'] == 1
fliter_2 = (alert_answer_custinf_1['sar_flag'] == 1)
alert_answer_custinf_answer1 = alert_answer_custinf_1[fliter_2]
print(alert_answer_custinf_answer1)

alert_answer_custinf_1['sar_flag'] == 2
fliter_3 = (alert_answer_custinf_1['sar_flag'] == 2)
alert_answer_custinf_answer2 = alert_answer_custinf_1[fliter_3]
print(alert_answer_custinf_answer2)

alert_answer_custinf_1['sar_flag'] == 3
fliter_4 = (alert_answer_custinf_1['sar_flag'] == 3)
alert_answer_custinf_answer3 = alert_answer_custinf_1[fliter_4]
print(alert_answer_custinf_answer3)


                                                cust_id  sar_flag
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1     002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40d...         0
2     002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...         0
3     002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...         0
4     0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...         0
...                                                 ...       ...
7259  ffe4dd95573f0eeacbaf8452f1db31ad927dddf41b17e7...         0
7260  ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...         0
7261  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...         0
7262  fff322436b556032e07690f12456d9eff3588c269d6210...         0
7263  fff94defcc33daab190e702926d583f6d805294354addb...         0

[7264 rows x 2 columns]
                                                cust_id  sar_flag
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1     002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40

**4.将合并好alert_answer_custinf_1按照每行顺序重复10000次  
10000次的选择原因是 在各表计数ID超过10000次的id很少**

In [18]:
#把合并好alert_answer_custinf_1按照每行顺序重复10000次
alert_answer_custinf_1_x10000=pd.DataFrame(np.repeat(alert_answer_custinf_1.values,10000,axis=0))
print(alert_answer_custinf_1_x10000.head())#没有标题
alert_answer_custinf_1_x10000.columns = ["cust_id","sar_flag"]#加标题
print(alert_answer_custinf_1_x10000)

                                                   0  1
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
                                                    cust_id sar_flag
0         0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
1         0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
2         0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
3         0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
4         0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
...                                                     ...      ...
72639995  fff94defcc33daab190e702926d583f6d805294354addb...        0
72639996  fff94defcc33daab190e702926d583f6d805294354addb...        0
72639997  fff94defcc33daab190e702926d583f6d

In [19]:
#确认是否有加倍成功
print("alert_answer_custinf_1\n",alert_answer_custinf_1.head(),"\n")  
print("alert_answer_custinf_1_x10000\n",alert_answer_custinf_1_x10000.head()) 
print("alert_answer_custinf_1",alert_answer_custinf_1.shape) 
print(alert_answer_custinf_1_x10000.shape)
row_1=alert_answer_custinf_1.shape[0]
row_2=alert_answer_custinf_1_x10000.shape[0]
print(row_2/row_1==10000)

alert_answer_custinf_1
                                              cust_id  sar_flag
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1  002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40d...         0
2  002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...         0
3  002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...         0
4  0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...         0 

alert_answer_custinf_1_x10000
                                              cust_id sar_flag
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
alert_answer_custinf_1 (7264, 2)
(72640000, 2)
True


In [20]:
#新增一列：作为索引指标，merge不支持多对一匹配，因此手动增加索引值标识
#同一个ID对应多个cust_id
alert_answer_custinf_1_x10000['rank'] =alert_answer_custinf_1_x10000['cust_id']
print(alert_answer_custinf_1_x10000.head())

                                             cust_id sar_flag  \
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0   
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0   
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0   
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0   
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0   

                                                rank  
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...  
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...  
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...  
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...  
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...  


In [21]:
#对同一个cust_id进行次数统计，结合ID作为索引指标
rank_1=alert_answer_custinf_1_x10000.groupby('rank').cumcount()
rank_1

0              0
1              1
2              2
3              3
4              4
            ... 
72639995    9995
72639996    9996
72639997    9997
72639998    9998
72639999    9999
Length: 72640000, dtype: int64

In [22]:
#把新的这列加入到原始data
alert_answer_custinf_1_x10000['rank'] =rank_1
print(alert_answer_custinf_1_x10000.head())
print(alert_answer_custinf_1_x10000.tail())

                                             cust_id sar_flag  rank
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     1
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     2
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     3
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     4
                                                    cust_id sar_flag  rank
72639995  fff94defcc33daab190e702926d583f6d805294354addb...        0  9995
72639996  fff94defcc33daab190e702926d583f6d805294354addb...        0  9996
72639997  fff94defcc33daab190e702926d583f6d805294354addb...        0  9997
72639998  fff94defcc33daab190e702926d583f6d805294354addb...        0  9998
72639999  fff94defcc33daab190e702926d583f6d805294354addb...        0  9999


**5.利用ID 使 ccba与sar值匹配  
由于一个ID有多个alert，且ID在子表多行的信息都不同，因此，只匹配这里面的ID 对应的所有的sar 为1 的次数**

In [23]:
print(train_x_ccba.head(),'\n')
print(train_x_ccba.tail(),'\n')
print(train_x_ccba['cust_id'].value_counts())#查看并计数-但这种写法不直观-用于观察需要重复基本表多少次才能完全匹配，merge不支持同一个指标出现多次
print(train_x_ccba.shape)

                                             cust_id  lupay   byymm   \
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -       -      
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        30    
2  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        61    
3  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        91    
4  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -       122    

      cycam   usgam   clamt   csamt   inamt   cucsm   cucah   
0   152,628     -       -       -       -       -       -     
1   152,628     -       -       -       -       -       -     
2   152,628     -       -       -       -       -       -     
3   152,628     -       -       -       -       -       -     
4   152,628     -       -       -       -       -       -      

                                                 cust_id    lupay   byymm   \
59070  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...      -       244    
59071  ffeb846f2294ec4397f80073

In [24]:
#对custinf的ID进行计数，便于补充alert_key-但是同一个ID多个alert_key时是无法补充这个ID对应的到底是哪一个alert值
train_x_ccba['rank'] =train_x_ccba['cust_id']
print(train_x_ccba.head(),'\n')
rank_2=train_x_ccba.groupby('rank').cumcount()
print(rank_2,'\n')
train_x_ccba['rank'] =rank_2
print(train_x_ccba.head(),'\n')
print(train_x_ccba.tail(),'\n')
print(train_x_ccba.shape,'\n')

                                             cust_id  lupay   byymm   \
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -       -      
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        30    
2  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        61    
3  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        91    
4  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -       122    

      cycam   usgam   clamt   csamt   inamt   cucsm   cucah   \
0   152,628     -       -       -       -       -       -      
1   152,628     -       -       -       -       -       -      
2   152,628     -       -       -       -       -       -      
3   152,628     -       -       -       -       -       -      
4   152,628     -       -       -       -       -       -      

                                                rank  
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...  
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...  
2  0172056578071e

In [25]:
#合并alert_answer_custinf_1_x10000与train_x_ccba，保留ccba的值
alert_answer_custinf_1_x10000_train_x_ccba=pd.merge(train_x_ccba,alert_answer_custinf_1_x10000,how = 'left',on = ['cust_id','rank'],sort = False)

In [26]:
print(alert_answer_custinf_1_x10000_train_x_ccba.head())
print(alert_answer_custinf_1_x10000_train_x_ccba.tail())
print(train_x_ccba.shape)
print("alert_answer_custinf_1_x10000_train_x_ccba",alert_answer_custinf_1_x10000_train_x_ccba.shape) 
train_x_ccba_answer=alert_answer_custinf_1_x10000_train_x_ccba
print(train_x_ccba_answer)

                                             cust_id  lupay   byymm   \
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -       -      
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        30    
2  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        61    
3  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -        91    
4  0172056578071e83399216fcd640bdc4de4583149d0fd9...    -       122    

      cycam   usgam   clamt   csamt   inamt   cucsm   cucah   rank sar_flag  
0   152,628     -       -       -       -       -       -        0      NaN  
1   152,628     -       -       -       -       -       -        1      NaN  
2   152,628     -       -       -       -       -       -        2      NaN  
3   152,628     -       -       -       -       -       -        3      NaN  
4   152,628     -       -       -       -       -       -        4      NaN  
                                                 cust_id    lupay   byymm   \
59070  ffeb846f2294ec

**6.利用ID 使 cusinf与sar值匹配  
刚刚是为了在answer里把ID加进去，现在是在custinf里加入answer**

In [27]:
#train_x_custinf同样的处理方式
print(train_x_custinf.head())
print(train_x_custinf.shape)
train_x_custinf['rank'] =train_x_custinf['cust_id']
print(train_x_custinf.head(),'\n')
rank_22=train_x_custinf.groupby('rank').cumcount()
print(rank_22,'\n')
train_x_custinf['rank'] =rank_22
print(train_x_custinf.head(),'\n')
print(train_x_custinf.tail(),'\n')
print(train_x_custinf.shape,'\n')

   alert_key                                            cust_id  risk_rank  \
0     352249  82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...          1   
1     352253  b212d14cb35676926682b2cf849e295d948888f556c07e...          1   
2     352254  e5b0002791c7852644a2730abeaa893cdf14a072ef7812...          1   
3     352280  74214c478dc6519fbefe4bc31693865bdcd698ab974b64...          3   
4     352282  0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...          1   

   occupation_code  total_asset  AGE  
0             19.0    1465816.0    7  
1              2.0      98177.0    2  
2             19.0    2052922.0    7  
3             15.0     201906.0    5  
4             12.0       7450.0    5  
(25751, 6)
   alert_key                                            cust_id  risk_rank  \
0     352249  82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...          1   
1     352253  b212d14cb35676926682b2cf849e295d948888f556c07e...          1   
2     352254  e5b0002791c7852644a2730abeaa893cdf14a0

In [28]:
alert_answer_custinf_1_x10000_train_x_custinf=pd.merge(train_x_custinf,alert_answer_custinf_1_x10000,how = 'left',on = ['cust_id','rank'],sort = False)
print(alert_answer_custinf_1_x10000_train_x_custinf.head())
print(alert_answer_custinf_1_x10000_train_x_custinf.tail())
print(train_x_custinf.shape)
print("alert_answer_custinf_1_x10000_train_x_custinf",alert_answer_custinf_1_x10000_train_x_custinf.shape) 
train_x_custinf_answer = alert_answer_custinf_1_x10000_train_x_custinf
print(train_x_custinf_answer)

   alert_key                                            cust_id  risk_rank  \
0     352249  82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...          1   
1     352253  b212d14cb35676926682b2cf849e295d948888f556c07e...          1   
2     352254  e5b0002791c7852644a2730abeaa893cdf14a072ef7812...          1   
3     352280  74214c478dc6519fbefe4bc31693865bdcd698ab974b64...          3   
4     352282  0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...          1   

   occupation_code  total_asset  AGE  rank sar_flag  
0             19.0    1465816.0    7     0        0  
1              2.0      98177.0    2     0        0  
2             19.0    2052922.0    7     0      NaN  
3             15.0     201906.0    5     0        0  
4             12.0       7450.0    5     0      NaN  
       alert_key                                            cust_id  \
25746     352123  b600c0720096c241f5aec16b096c6a353492eee8a5855a...   
25747     352124  06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a.

**7.利用ID 使 cusinf与sar值匹配  
刚刚是为了在answer里把ID加进去，现在是在custinf里加入answer**

In [29]:
#train_x_remit1同样的处理方式
print(train_x_remit1.head())
print(train_x_remit1.shape)
train_x_remit1['rank'] =train_x_remit1['cust_id']
print(train_x_remit1.head(),'\n')
rank_222=train_x_remit1.groupby('rank').cumcount()
print(rank_222,'\n')
train_x_remit1['rank'] =rank_222
print(train_x_remit1.head(),'\n')
print(train_x_remit1.tail(),'\n')
print(train_x_remit1.shape,'\n')

                                             cust_id  trans_date  trans_no  \
0  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         148         0   
1  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         165         0   
2  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         231         0   
3  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         291         0   
4  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         389         0   

   trade_amount_usd  
0          100528.0  
1          100528.0  
2           89479.0  
3          100528.0  
4           30158.0  
(17167, 4)
                                             cust_id  trans_date  trans_no  \
0  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         148         0   
1  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         165         0   
2  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         231         0   
3  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         291         0  

In [30]:
alert_answer_custinf_1_x10000_train_x_remit1=pd.merge(train_x_remit1,alert_answer_custinf_1_x10000,how = 'left',on = ['cust_id','rank'],sort = False)
print(alert_answer_custinf_1_x10000_train_x_remit1.head())
print(alert_answer_custinf_1_x10000_train_x_remit1.tail())
print(train_x_remit1.shape)
print("alert_answer_custinf_1_x10000_train_x_remit1",alert_answer_custinf_1_x10000_train_x_remit1.shape) 
train_x_remit1_answer = alert_answer_custinf_1_x10000_train_x_remit1
print(train_x_remit1_answer)

                                             cust_id  trans_date  trans_no  \
0  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         148         0   
1  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         165         0   
2  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         231         0   
3  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         291         0   
4  01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...         389         0   

   trade_amount_usd  rank sar_flag  
0          100528.0     0        0  
1          100528.0     1        0  
2           89479.0     2        0  
3          100528.0     3        0  
4           30158.0     4        0  
                                                 cust_id  trans_date  \
17162  ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...         244   
17163  ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...         284   
17164  ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...         298   
17165  ff6c1217b5e039

**8.处理train_x_cdtx0001  
分拆ID次数在10000次以上和10000次以下  
10000内使用 alert_answer_custinf_1_x10000 匹配  
10000上使用 alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000 匹配**

In [31]:
print(train_x_cdtx0001.head())
print(train_x_cdtx0001.shape)

                                             cust_id  date  country  cur_type  \
0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...     1      130        47   
1  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
2  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
3  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    13      130        47   
4  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    15      130        47   

      amt  
0  2312.0  
1   808.0  
2  7099.0  
3   306.0  
4   255.0  
(1043014, 5)


（1）找到ID次数在10000以上和以下的ID账号  
总计cdtx0001 ID 3945  
总计cdtx0001 ID 10000内 3941  
总计cdtx0001 ID 10000上 4  

In [32]:
#对重复次数在10000以上及一下进行分拆
#train_x_cdtx0001 ID次数 在10000内 的ID 储存在 train_x_cdtx0001_countid_9999
train_x_cdtx0001_countid_9999=train_x_cdtx0001_countid[train_x_cdtx0001_countid["amt"] < 10000]
train_x_cdtx0001_countid_9999=train_x_cdtx0001_countid_9999["cust_id"]
train_x_cdtx0001_countid_9999=pd.DataFrame(train_x_cdtx0001_countid_9999)
train_x_cdtx0001_countid_9999.insert(1,'countid',0)
print(train_x_cdtx0001_countid_9999)
#train_x_cdtx0001 ID次数 在10000上 的ID 储存在 train_x_cdtx0001_countid_10000
train_x_cdtx0001_countid_10000=train_x_cdtx0001_countid[train_x_cdtx0001_countid["amt"] >= 10000]
train_x_cdtx0001_countid_10000=train_x_cdtx0001_countid_10000["cust_id"]
train_x_cdtx0001_countid_10000=pd.DataFrame(train_x_cdtx0001_countid_10000)
train_x_cdtx0001_countid_10000.insert(1,'countid',1)
print(train_x_cdtx0001_countid_10000)
#确认分类数据量匹配
print(train_x_cdtx0001_countid.shape)
print(train_x_cdtx0001_countid_9999.shape)
print(train_x_cdtx0001_countid_10000.shape)

                                                cust_id  countid
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
1     002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...        0
2     002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...        0
3     0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...        0
4     0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...        0
...                                                 ...      ...
3940  ffa5bcc6f88df5c33e809da4daa54de0031803f37d8f11...        0
3941  ffae7fef923c78298cd647a3d573e48cc1404b07ac794a...        0
3942  ffdbc1a5135cb6feddf7355e5656abf6973f21639c7b9b...        0
3943  ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...        0
3944  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...        0

[3941 rows x 2 columns]
                                                cust_id  countid
1001  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...        1
1782  75945c8b1b7af88340ffe4169b39341dfded2c07248d36...        1


（2）ID数在 10000内  的ID重复10000次，并赋予索引指标rank--为了能在原始data中标记是否超过10000次

In [33]:
#按照每行顺序重复10000次
train_x_cdtx0001_countid_9999_x10000=pd.DataFrame(np.repeat(train_x_cdtx0001_countid_9999.values,10000,axis=0))
print(train_x_cdtx0001_countid_9999_x10000.head())#没有标题
train_x_cdtx0001_countid_9999_x10000.columns = ["cust_id","countid"]#加标题
print(train_x_cdtx0001_countid_9999_x10000)
#确认是否有加倍成功
print("train_x_cdtx0001_countid_9999\n",train_x_cdtx0001_countid_9999.head(),"\n")  
print("train_x_cdtx0001_countid_9999_x10000\n",train_x_cdtx0001_countid_9999_x10000.head()) 
print("train_x_cdtx0001_countid_9999",train_x_cdtx0001_countid_9999.shape) 
print(train_x_cdtx0001_countid_9999_x10000.shape)
row_11=train_x_cdtx0001_countid_9999.shape[0]
row_22=train_x_cdtx0001_countid_9999_x10000.shape[0]
print(row_22/row_11==10000)
train_x_cdtx0001_countid_9999_x10000['rank'] =train_x_cdtx0001_countid_9999_x10000['cust_id']
print(train_x_cdtx0001_countid_9999_x10000.head())
#对同一个cust_id进行次数统计，结合ID作为索引指标
rank_11=train_x_cdtx0001_countid_9999_x10000.groupby('rank').cumcount()
print(rank_11)
#把新的这列加入到原始data
train_x_cdtx0001_countid_9999_x10000['rank'] =rank_11
print(train_x_cdtx0001_countid_9999_x10000.head())
print(train_x_cdtx0001_countid_9999_x10000.tail())

                                                   0  1
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
                                                    cust_id countid
0         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
1         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
2         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
3         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
4         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
...                                                     ...     ...
39409995  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...       0
39409996  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...       0
39409997  ffeb846f2294ec4397f800736dc1e14eb754da5425

（3）将ID数10000内的ID和countid值加入原始data中，确认哪些是超过10000的，并标记为0

In [34]:
#对train_x_cdtx0001 ID进行计数
train_x_cdtx0001['rank'] =train_x_cdtx0001['cust_id']
print(train_x_cdtx0001.head(),'\n')
rank_2222=train_x_cdtx0001.groupby('rank').cumcount()
print(rank_2222,'\n')
train_x_cdtx0001['rank'] =rank_2222
print(train_x_cdtx0001.head(),'\n')
print(train_x_cdtx0001.tail(),'\n')
print(train_x_cdtx0001.shape,'\n')

                                             cust_id  date  country  cur_type  \
0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...     1      130        47   
1  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
2  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
3  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    13      130        47   
4  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    15      130        47   

      amt                                               rank  
0  2312.0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...  
1   808.0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...  
2  7099.0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...  
3   306.0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...  
4   255.0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...   

0           0
1           1
2           2
3           3
4           4
           ..
1043009    50
1043010    51
1043011    52
1043012

In [35]:
train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001=pd.merge(train_x_cdtx0001,train_x_cdtx0001_countid_9999_x10000,how = 'left',on = ['cust_id','rank'],sort = False)
print(train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001.head())
print(train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001.tail())
print("train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001",train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001.shape) 

                                             cust_id  date  country  cur_type  \
0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...     1      130        47   
1  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
2  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
3  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    13      130        47   
4  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    15      130        47   

      amt  rank countid  
0  2312.0     0       0  
1   808.0     1       0  
2  7099.0     2       0  
3   306.0     3       0  
4   255.0     4       0  
                                                   cust_id  date  country  \
1043009  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   359      130   
1043010  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   360      130   
1043011  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   361      130   
1043012  ffeb846f2294ec4397f800736dc1e14eb754da54

（4）将原始data中 ID数将超过10000次的标记为1

In [36]:
print(train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001[train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001["countid"] != 0])
values = {'countid':'1'}
train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001.fillna(value = values,inplace = True)
print(train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001[train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001["countid"] != 0])
print(train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001)

                                                  cust_id  date  country  \
44113   75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     0      125   
44114   75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     3      125   
44115   75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     2      135   
44116   75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     2      135   
44117   75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     4      135   
...                                                   ...   ...      ...   
869681  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   370      130   
869682  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   381      130   
869683  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   389      130   
869684  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   392      130   
869685  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   393      130   

        cur_type      amt   rank countid  
44113         47    248.0      0     NaN  
4

（5）通过countid 为 0 或 1分拆 ID次数出现10000次以上或内

In [37]:
train_x_cdtx0001_0 =train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001[train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001["countid"] == 0]
train_x_cdtx0001_1 =train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001[train_x_cdtx0001_countid_9999_x10000_train_x_cdtx0001["countid"] != 0]

print(train_x_cdtx0001_0)
print(train_x_cdtx0001_1)

                                                   cust_id  date  country  \
0        0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...     1      130   
1        0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130   
2        0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130   
3        0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    13      130   
4        0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    15      130   
...                                                    ...   ...      ...   
1043009  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   359      130   
1043010  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   360      130   
1043011  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   361      130   
1043012  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   364      130   
1043013  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   377       53   

         cur_type     amt  rank countid  
0              47  2312.0     0  

（6）对于ID出现次数小于10000的，直接用alert_answer_custinf_1_x10000 匹配sar值

In [38]:
alert_answer_custinf_1_x10000_train_x_cdtx0001_0=pd.merge(train_x_cdtx0001_0,alert_answer_custinf_1_x10000,how = 'left',on = ['cust_id','rank'],sort = False)
print(alert_answer_custinf_1_x10000_train_x_cdtx0001_0)

                                                  cust_id  date  country  \
0       0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...     1      130   
1       0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130   
2       0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130   
3       0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    13      130   
4       0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    15      130   
...                                                   ...   ...      ...   
876083  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   359      130   
876084  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   360      130   
876085  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   361      130   
876086  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   364      130   
876087  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...   377       53   

        cur_type     amt  rank countid sar_flag  
0             47  2312.0     0       

（6）对于ID出现次数大于等于10000的   
首先将这些ID在alert_answer_custinf_1中找到及其sar值  
然后将这些值重复15万次   
再将10000万次以上的ID 与15万次匹配--只有4个ID  

In [39]:
print(alert_answer_custinf_1)
print(alert_answer_custinf_1[alert_answer_custinf_1["sar_flag"] == 0])
print(alert_answer_custinf_1[alert_answer_custinf_1["sar_flag"] != 0])
print(train_x_cdtx0001_countid_10000)
alert_answer_custinf_1_train_x_cdtx0001_countid_10000=pd.merge(train_x_cdtx0001_countid_10000,alert_answer_custinf_1,how = 'left',on = ['cust_id'],sort = False)
alert_answer_custinf_1_train_x_cdtx0001_countid_10000=alert_answer_custinf_1_train_x_cdtx0001_countid_10000.drop(columns = ['countid'])
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000)

                                                cust_id  sar_flag
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1     002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40d...         0
2     002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...         0
3     002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...         0
4     0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...         0
...                                                 ...       ...
7259  ffe4dd95573f0eeacbaf8452f1db31ad927dddf41b17e7...         0
7260  ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...         0
7261  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...         0
7262  fff322436b556032e07690f12456d9eff3588c269d6210...         0
7263  fff94defcc33daab190e702926d583f6d805294354addb...         0

[7264 rows x 2 columns]
                                                cust_id  sar_flag
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1     002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40

In [40]:
#按照每行顺序重复150000次
alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000=pd.DataFrame(np.repeat(alert_answer_custinf_1_train_x_cdtx0001_countid_10000.values,150000,axis=0))
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.head())#没有标题
alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.columns = ["cust_id","sar_flag"]#加标题
alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000
#确认是否有加倍成功
print("alert_answer_custinf_1_train_x_cdtx0001_countid_10000\n",alert_answer_custinf_1_train_x_cdtx0001_countid_10000.head(),"\n")  
print("alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000\n",alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.head()) 
print("alert_answer_custinf_1_train_x_cdtx0001_countid_10000",alert_answer_custinf_1_train_x_cdtx0001_countid_10000.shape) 
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.shape)
row_11=alert_answer_custinf_1_train_x_cdtx0001_countid_10000.shape[0]
row_22=alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.shape[0]
print(row_22/row_11==10000)
alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000['rank'] =alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000['cust_id']
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.head())
#对同一个cust_id进行次数统计，结合ID作为索引指标
rank_111=alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.groupby('rank').cumcount()
print(rank_111)
#把新的这列加入到原始data
alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000['rank'] =rank_111
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.head())
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.tail())

                                                   0  1
0  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...  1
1  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...  1
2  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...  1
3  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...  1
4  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...  1
alert_answer_custinf_1_train_x_cdtx0001_countid_10000
                                              cust_id  sar_flag
0  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...         1
1  75945c8b1b7af88340ffe4169b39341dfded2c07248d36...         0
2  77eb881fb8ad47743816447a9c9f9712f64b1e42bf56fd...         1
3  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...         0 

alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000
                                              cust_id sar_flag
0  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...        1
1  4220624de6d38bd4b41d1c32932e03f3c0e85b1b08b7e3...        1
2  4220624de6d38bd4b41d1c32932e03f3c0e85b1b

In [41]:
alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000_train_x_cdtx0001_1=pd.merge(train_x_cdtx0001_1,alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000,how = 'left',on = ['cust_id','rank'],sort = False)
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000_train_x_cdtx0001_1)

                                                  cust_id  date  country  \
0       75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     0      125   
1       75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     3      125   
2       75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     2      135   
3       75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     2      135   
4       75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     4      135   
...                                                   ...   ...      ...   
166921  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   370      130   
166922  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   381      130   
166923  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   389      130   
166924  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   392      130   
166925  c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...   393      130   

        cur_type      amt   rank countid sar_flag  
0             47    248.0      0   

（7）合并分别匹配的1000次内及上的cdtx0001 表

In [42]:
print(alert_answer_custinf_1_x10000_train_x_cdtx0001_0.head())
print(alert_answer_custinf_1_x10000_train_x_cdtx0001_0.shape)
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000_train_x_cdtx0001_1.head())
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000_train_x_cdtx0001_1.shape)
frames_train_x_cdtx0001 = [alert_answer_custinf_1_x10000_train_x_cdtx0001_0,alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000_train_x_cdtx0001_1]
train_x_cdtx0001_answer = pd.concat(frames_train_x_cdtx0001)
print(train_x_cdtx0001_answer)

                                             cust_id  date  country  cur_type  \
0  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...     1      130        47   
1  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
2  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    10      130        47   
3  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    13      130        47   
4  0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...    15      130        47   

      amt  rank countid sar_flag  
0  2312.0     0       0      NaN  
1   808.0     1       0      NaN  
2  7099.0     2       0      NaN  
3   306.0     3       0      NaN  
4   255.0     4       0      NaN  
(876088, 8)
                                             cust_id  date  country  cur_type  \
0  75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     0      125        47   
1  75945c8b1b7af88340ffe4169b39341dfded2c07248d36...     3      125        47   
2  75945c8b1b7af88340ffe4169b39341dfded2c07248d3

**9.处理train_x_dp  
分拆ID次数在10000次以上和10000次以下  
10000内使用 alert_answer_custinf_1_x10000 匹配  
10000上使用 alert_answer_custinf_1_train_x_dp_countid_10000_x150000 匹配**

In [43]:
#对重复次数在10000以上及一下进行分拆
train_x_dp_countid_9999=train_x_dp_countid[train_x_dp_countid["ATM"] < 10000]
train_x_dp_countid_9999=train_x_dp_countid_9999["cust_id"]
train_x_dp_countid_9999=pd.DataFrame(train_x_dp_countid_9999)
train_x_dp_countid_9999.insert(1,'countid',0)
print(train_x_dp_countid_9999)
train_x_dp_countid_10000=train_x_dp_countid[train_x_dp_countid["ATM"] >= 10000]
train_x_dp_countid_10000=train_x_dp_countid_10000["cust_id"]
train_x_dp_countid_10000=pd.DataFrame(train_x_dp_countid_10000)
train_x_dp_countid_10000.insert(1,'countid',1)
print(train_x_dp_countid_10000)
print(train_x_dp_countid.shape)
print(train_x_dp_countid_9999.shape)
print(train_x_dp_countid_10000.shape)

                                                cust_id  countid
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...        0
1     002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...        0
2     002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...        0
3     0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...        0
4     0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...        0
...                                                 ...      ...
6191  ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...        0
6192  ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...        0
6193  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...        0
6194  fff322436b556032e07690f12456d9eff3588c269d6210...        0
6195  fff94defcc33daab190e702926d583f6d805294354addb...        0

[6183 rows x 2 columns]
                                                cust_id  countid
34    01c705b457297805c1c08137cb3adb164b8f393c265db2...        1
354   0fd01fa104450a708a61b529afcbcc735b1ccb3e7a1581...        1


In [44]:
#按照每行顺序重复10000次
train_x_dp_countid_9999_x10000=pd.DataFrame(np.repeat(train_x_dp_countid_9999.values,10000,axis=0))
print(train_x_dp_countid_9999_x10000.head())#没有标题
train_x_dp_countid_9999_x10000.columns = ["cust_id","countid"]#加标题
print(train_x_dp_countid_9999_x10000)
#确认是否有加倍成功
print("train_x_dp_countid_9999\n",train_x_dp_countid_9999.head(),"\n")  
print("train_x_dp_countid_9999_x10000\n",train_x_dp_countid_9999_x10000.head()) 
print("train_x_dp_countid_9999",train_x_dp_countid_9999.shape) 
print(train_x_dp_countid_9999_x10000.shape)
row_111=train_x_dp_countid_9999.shape[0]
row_222=train_x_dp_countid_9999_x10000.shape[0]
print(row_222/row_111==10000)
train_x_dp_countid_9999_x10000['rank'] =train_x_dp_countid_9999_x10000['cust_id']
print(train_x_dp_countid_9999_x10000.head())
#对同一个cust_id进行次数统计，结合ID作为索引指标
rank_111=train_x_dp_countid_9999_x10000.groupby('rank').cumcount()
print(rank_111)
#把新的这列加入到原始data
train_x_dp_countid_9999_x10000['rank'] =rank_111
print(train_x_dp_countid_9999_x10000.head())
print(train_x_dp_countid_9999_x10000.tail())

                                                   0  1
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...  0
                                                    cust_id countid
0         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
1         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
2         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
3         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
4         0015cc01f553512007705edf855baa16f1b57a16fb107e...       0
...                                                     ...     ...
61829995  fff94defcc33daab190e702926d583f6d805294354addb...       0
61829996  fff94defcc33daab190e702926d583f6d805294354addb...       0
61829997  fff94defcc33daab190e702926d583f6d805294354

In [45]:
#对ID进行计数，便于补充alert_key-但是同一个ID多个alert_key时是无法补充这个ID对应的到底是哪一个alert值
train_x_dp['rank'] =train_x_dp['cust_id']
print(train_x_dp.head(),'\n')
rank_22222=train_x_dp.groupby('rank').cumcount()
print(rank_22222,'\n')
train_x_dp['rank'] =rank_22222
print(train_x_dp.head(),'\n')
print(train_x_dp.tail(),'\n')
print(train_x_dp.shape,'\n')

                                             cust_id debit_credit  tx_date  \
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       36   
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       42   
2  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       39   
3  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       49   
4  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       70   

   tx_time  tx_type    tx_amt  exchg_rate  info_asset_code  fiscTxId  \
0       18        2   68265.0         1.0               16       NaN   
1       17        2  932058.0         1.0               16       NaN   
2       18        2    6089.0         1.0               16       NaN   
3       15        2  776715.0         1.0               16       NaN   
4       19        2   61630.0         1.0               16       NaN   

   txbranch  cross_bank  ATM  \
0       NaN           0    0   
1       NaN           0    0   
2 

In [46]:
train_x_dp_countid_9999_x10000_train_x_dp=pd.merge(train_x_dp,train_x_dp_countid_9999_x10000,how = 'left',on = ['cust_id','rank'],sort = False)
print(train_x_dp_countid_9999_x10000_train_x_dp.head())
print(train_x_dp_countid_9999_x10000_train_x_dp.tail())
print("train_x_dp_countid_9999_x10000_train_x_dp",train_x_dp_countid_9999_x10000_train_x_dp.shape) 

                                             cust_id debit_credit  tx_date  \
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       36   
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       42   
2  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       39   
3  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       49   
4  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       70   

   tx_time  tx_type    tx_amt  exchg_rate  info_asset_code  fiscTxId  \
0       18        2   68265.0         1.0               16       NaN   
1       17        2  932058.0         1.0               16       NaN   
2       18        2    6089.0         1.0               16       NaN   
3       15        2  776715.0         1.0               16       NaN   
4       19        2   61630.0         1.0               16       NaN   

   txbranch  cross_bank  ATM  rank countid  
0       NaN           0    0     0       0  
1       

In [47]:
print(train_x_dp_countid_9999_x10000_train_x_dp[train_x_dp_countid_9999_x10000_train_x_dp["countid"] != 0])
values = {'countid':'1'}
train_x_dp_countid_9999_x10000_train_x_dp.fillna(value = values,inplace = True)
print(train_x_dp_countid_9999_x10000_train_x_dp[train_x_dp_countid_9999_x10000_train_x_dp["countid"] != 0])
print(train_x_dp_countid_9999_x10000_train_x_dp)

                                                   cust_id debit_credit  \
259      01c705b457297805c1c08137cb3adb164b8f393c265db2...           DB   
260      01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
261      01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
262      01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
263      01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
...                                                    ...          ...   
1898389  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           CR   
1898390  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           CR   
1898391  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           DB   
1898392  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           DB   
1898393  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           CR   

         tx_date  tx_time  tx_type   tx_amt  exchg_rate  info_asset_code  \
259          323       

In [48]:
train_x_dp_0 =train_x_dp_countid_9999_x10000_train_x_dp[train_x_dp_countid_9999_x10000_train_x_dp["countid"] == 0]
train_x_dp_1 =train_x_dp_countid_9999_x10000_train_x_dp[train_x_dp_countid_9999_x10000_train_x_dp["countid"] != 0]

print(train_x_dp_0)
print(train_x_dp_1)

                                                   cust_id debit_credit  \
0        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
1        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
2        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
3        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
4        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
...                                                    ...          ...   
1969813  fff94defcc33daab190e702926d583f6d805294354addb...           CR   
1969814  fff94defcc33daab190e702926d583f6d805294354addb...           CR   
1969815  fff94defcc33daab190e702926d583f6d805294354addb...           CR   
1969816  fff94defcc33daab190e702926d583f6d805294354addb...           DB   
1969817  fff94defcc33daab190e702926d583f6d805294354addb...           DB   

         tx_date  tx_time  tx_type    tx_amt  exchg_rate  info_asset_code  \
0             36      

In [49]:
alert_answer_custinf_1_x10000_train_x_dp_0=pd.merge(train_x_dp_0,alert_answer_custinf_1_x10000,how = 'left',on = ['cust_id','rank'],sort = False)
print(alert_answer_custinf_1_x10000_train_x_dp_0)

                                                   cust_id debit_credit  \
0        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
1        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
2        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
3        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
4        0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR   
...                                                    ...          ...   
1517156  fff94defcc33daab190e702926d583f6d805294354addb...           CR   
1517157  fff94defcc33daab190e702926d583f6d805294354addb...           CR   
1517158  fff94defcc33daab190e702926d583f6d805294354addb...           CR   
1517159  fff94defcc33daab190e702926d583f6d805294354addb...           DB   
1517160  fff94defcc33daab190e702926d583f6d805294354addb...           DB   

         tx_date  tx_time  tx_type    tx_amt  exchg_rate  info_asset_code  \
0             36      

In [50]:
print(alert_answer_custinf_1)
print(alert_answer_custinf_1[alert_answer_custinf_1["sar_flag"] == 0])
print(alert_answer_custinf_1[alert_answer_custinf_1["sar_flag"] != 0])
print(train_x_dp_countid_10000)
alert_answer_custinf_1_train_x_dp_countid_10000=pd.merge(train_x_dp_countid_10000,alert_answer_custinf_1,how = 'left',on = ['cust_id'],sort = False)
alert_answer_custinf_1_train_x_dp_countid_10000=alert_answer_custinf_1_train_x_dp_countid_10000.drop(columns = ['countid'])
print(alert_answer_custinf_1_train_x_dp_countid_10000)

                                                cust_id  sar_flag
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1     002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40d...         0
2     002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...         0
3     002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...         0
4     0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...         0
...                                                 ...       ...
7259  ffe4dd95573f0eeacbaf8452f1db31ad927dddf41b17e7...         0
7260  ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...         0
7261  ffeb846f2294ec4397f800736dc1e14eb754da5425398d...         0
7262  fff322436b556032e07690f12456d9eff3588c269d6210...         0
7263  fff94defcc33daab190e702926d583f6d805294354addb...         0

[7264 rows x 2 columns]
                                                cust_id  sar_flag
0     0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1     002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40

In [51]:
#按照每行顺序重复150000次
alert_answer_custinf_1_train_x_dp_countid_10000_x150000=pd.DataFrame(np.repeat(alert_answer_custinf_1_train_x_dp_countid_10000.values,150000,axis=0))
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000.head())#没有标题
alert_answer_custinf_1_train_x_dp_countid_10000_x150000.columns = ["cust_id","sar_flag"]#加标题
alert_answer_custinf_1_train_x_dp_countid_10000_x150000
#确认是否有加倍成功
print("alert_answer_custinf_1_train_x_dp_countid_10000\n",alert_answer_custinf_1_train_x_dp_countid_10000.head(),"\n")  
print("alert_answer_custinf_1_train_x_dp_countid_10000_x150000\n",alert_answer_custinf_1_train_x_dp_countid_10000_x150000.head()) 
print("alert_answer_custinf_1_train_x_dp_countid_10000",alert_answer_custinf_1_train_x_dp_countid_10000.shape) 
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000.shape)
row_111=alert_answer_custinf_1_train_x_dp_countid_10000.shape[0]
row_222=alert_answer_custinf_1_train_x_dp_countid_10000_x150000.shape[0]
print(row_222/row_111==10000)
alert_answer_custinf_1_train_x_dp_countid_10000_x150000['rank'] =alert_answer_custinf_1_train_x_dp_countid_10000_x150000['cust_id']
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000.head())
#对同一个cust_id进行次数统计，结合ID作为索引指标
rank_111=alert_answer_custinf_1_train_x_dp_countid_10000_x150000.groupby('rank').cumcount()
print(rank_111)
#把新的这列加入到原始data
alert_answer_custinf_1_train_x_dp_countid_10000_x150000['rank'] =rank_111
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000.head())
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000.tail())

                                                   0  1
0  01c705b457297805c1c08137cb3adb164b8f393c265db2...  0
1  01c705b457297805c1c08137cb3adb164b8f393c265db2...  0
2  01c705b457297805c1c08137cb3adb164b8f393c265db2...  0
3  01c705b457297805c1c08137cb3adb164b8f393c265db2...  0
4  01c705b457297805c1c08137cb3adb164b8f393c265db2...  0
alert_answer_custinf_1_train_x_dp_countid_10000
                                              cust_id  sar_flag
0  01c705b457297805c1c08137cb3adb164b8f393c265db2...         0
1  0fd01fa104450a708a61b529afcbcc735b1ccb3e7a1581...         0
2  11eb3723249ee2e8415a950bc3f7b52eef5baec604855b...         0
3  12c9e6d35500d2a96fc2b22a9da8e3deb6048de515a16e...         1
4  32c290607487b2163ec1d12846ecf4bb022659ae004fd6...         0 

alert_answer_custinf_1_train_x_dp_countid_10000_x150000
                                              cust_id sar_flag
0  01c705b457297805c1c08137cb3adb164b8f393c265db2...        0
1  01c705b457297805c1c08137cb3adb164b8f393c265db2...  

In [52]:
alert_answer_custinf_1_train_x_dp_countid_10000_x150000_train_x_dp_1=pd.merge(train_x_dp_1,alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000,how = 'left',on = ['cust_id','rank'],sort = False)
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000_train_x_dp_1)

                                                  cust_id debit_credit  \
0       01c705b457297805c1c08137cb3adb164b8f393c265db2...           DB   
1       01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
2       01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
3       01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
4       01c705b457297805c1c08137cb3adb164b8f393c265db2...           CR   
...                                                   ...          ...   
452652  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           CR   
452653  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           CR   
452654  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           DB   
452655  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           DB   
452656  f238341722c3de6abb5b91d1a7ca9a7e1c3b8ca1726e2d...           CR   

        tx_date  tx_time  tx_type   tx_amt  exchg_rate  info_asset_code  \
0           323       10        2  1

In [53]:
print(alert_answer_custinf_1_x10000_train_x_dp_0.head())
print(alert_answer_custinf_1_x10000_train_x_dp_0.shape)
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000_train_x_dp_1.head())
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000_train_x_dp_1.shape)
frames_train_x_dp = [alert_answer_custinf_1_x10000_train_x_dp_0,alert_answer_custinf_1_train_x_dp_countid_10000_x150000_train_x_dp_1]
train_x_dp_answer = pd.concat(frames_train_x_dp)
print(train_x_dp_answer)

                                             cust_id debit_credit  tx_date  \
0  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       36   
1  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       42   
2  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       39   
3  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       49   
4  0172056578071e83399216fcd640bdc4de4583149d0fd9...           CR       70   

   tx_time  tx_type    tx_amt  exchg_rate  info_asset_code  fiscTxId  \
0       18        2   68265.0         1.0               16       NaN   
1       17        2  932058.0         1.0               16       NaN   
2       18        2    6089.0         1.0               16       NaN   
3       15        2  776715.0         1.0               16       NaN   
4       19        2   61630.0         1.0               16       NaN   

   txbranch  cross_bank  ATM  rank countid sar_flag  
0       NaN           0    0     0       0  

**10.所有匹配好sar值的表**

In [54]:
print(alert_answer_custinf_1.head())
print(alert_answer_custinf_1_x10000.head())
print(train_x_cdtx0001_countid_9999_x10000.head())
print(alert_answer_custinf_1_train_x_cdtx0001_countid_10000_x150000.head())
print(train_x_dp_countid_9999_x10000.head())
print(alert_answer_custinf_1_train_x_dp_countid_10000_x150000.head())

print(train_x_ccba_answer.head())
print(train_x_custinf_answer.head())
print(train_x_remit1_answer.head())
print(train_x_cdtx0001_answer.head())
print(train_x_dp_answer.head())

                                             cust_id  sar_flag
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...         0
1  002630daf48bb6a37c7fb0c83f201f8d5bb9862283e40d...         0
2  002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...         0
3  002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...         0
4  0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...         0
                                             cust_id sar_flag  rank
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     1
2  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     2
3  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     3
4  0015cc01f553512007705edf855baa16f1b57a16fb107e...        0     4
                                             cust_id countid  rank
0  0015cc01f553512007705edf855baa16f1b57a16fb107e...       0     0
1  0015cc01f553512007705edf855baa16f1b57a16fb107e...       0     1
2  0015cc01f5

In [55]:
outputpath_1='train_x_ccba_answer.csv'
train_x_ccba_answer.to_csv(outputpath_1,sep=',',index=False,header=True)

outputpath_2='train_x_custinf_answer.csv'
train_x_custinf_answer.to_csv(outputpath_2,sep=',',index=False,header=True)

outputpath_3='train_x_remit1_answer.csv'
train_x_remit1_answer.to_csv(outputpath_3,sep=',',index=False,header=True)

outputpath_4='train_x_cdtx0001_answer.csv'
train_x_cdtx0001_answer.to_csv(outputpath_4,sep=',',index=False,header=True)

outputpath_5='train_x_dp_answer.csv'
train_x_dp_answer.to_csv(outputpath_5,sep=',',index=False,header=True)